In [3]:
#https://github.com/garywu/google-compute-engine-selenium
# install chromium, its driver, and selenium
# !sudo apt install chromium-chromedriver
# !sudo cp /usr/lib/chromium-browser/chromedriver /usr/bin
# !pip install selenium
# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver= webdriver.Chrome('chromedriver',options=options)


# open it, go to a website, and get results
# wd = webdriver.Chrome('chromedriver',options=options)
# wd.get("https://www.website.com")
# print(wd.page_source)  # results

In [2]:
import csv


f = open("kappah.csv", "w", encoding='utf-8')
writer = csv.DictWriter(f, fieldnames=["name","brand","gender","category","size","price","color","url","img_url","article_no","blob_url"])
writer.writeheader()
f.close()

import pandas as pd
df=pd.read_csv("kappah.csv", encoding='utf-8')
df.head()

,name,brand,gender,category,size,price,color,url,img_url,article_no,blob_url


In [4]:
from google.cloud import storage

def upload_to_bucket(blob_name, path_to_file):
    """ Upload data to a bucket""" #doc string in function

    # Explicitly use service account credentials by specifying the private key
    # file.
    storage_client = storage.Client.from_service_account_json(
        'auth.json')

    #print(buckets = list(storage_client.list_buckets())

    bucket = storage_client.get_bucket('urbankiddo')
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(path_to_file)

    #returns a public url
    return blob.public_url

In [5]:
import urllib.request
import csv

import time
import random
error=[]
def scrapper(main_url,gender):
    driver= webdriver.Chrome('chromedriver',options=options)
    driver.get(main_url)
    driver.set_window_size(1920,1080)

    xpath='//*[@id="form1"]/div[3]/div/main/div[6]/div[4]/div[1]/ul/li/ul/li[2]/ul/li'
    categ = driver.find_elements_by_xpath(xpath)
    categ.pop(0)
    categ.pop(0)
    categorys=[]
    sub_urls=[]
    for c in categ:
        categorys.append(c.find_element_by_xpath(".//a").get_attribute('innerHTML'))
        sub_urls.append(c.find_element_by_xpath(".//a").get_attribute('href'))
    driver.quit()
        
    for category,sub_url in zip(categorys,sub_urls):
        driver= webdriver.Chrome('chromedriver',options=options)
        driver.get(sub_url)
        driver.set_window_size(1920,1080)
        
        wait = WebDriverWait(driver, 10)
        productCountPresent = ec.presence_of_element_located(
            (By.CLASS_NAME, 'product-card__url'))
        wait.until(productCountPresent)
        
        
                
        wait = WebDriverWait(driver, 10)
        button_clickable = ec.presence_of_element_located(
            (By.CLASS_NAME, 'filterDropdown__button'))
        wait.until(button_clickable)
        
                   
        elem=driver.find_element_by_xpath('//div[@class="filterDropdown__wrapper"]//button[@class="filterDropdown__button"]')
        driver.execute_script("$(arguments[0]).click();", elem)

        element=driver.find_element_by_xpath('//div[@class="filterDropdown__wrapper"]')
        
        wait = WebDriverWait(driver, 10)
        checkbox_clickable = ec.presence_of_element_located(
            (By.XPATH, '//div[@class="filterDropdown__wrapper"]//input'))
        wait.until(checkbox_clickable)
        
        
        for i in element.find_elements_by_xpath('.//input'):            
            print(i.get_attribute('outerHTML'))
        if gender=='boy':
            ele2=element.find_elements_by_xpath('.//input')
            driver.execute_script("(arguments[0]).click();", ele2[0])    #
        elif gender=='girl':
            ele2=element.find_elements_by_xpath('.//input')    #
            driver.execute_script("(arguments[0]).click();", ele2[1])    #

        
        ### scrolldown()
        SCROLL_PAUSE_TIME = 0.5


        # Get scroll height.
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:

            # Scroll down to the bottom.
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load the page.
            time.sleep(2)

            # Calculate new scroll height and compare with last scroll height.
            new_height = driver.execute_script("return document.body.scrollHeight")

            if new_height == last_height:

                break

            last_height = new_height
            try:
                wait = WebDriverWait(driver, 5)
                productCountPresent = ec.presence_of_element_located(
                (By.CLASS_NAME, 'product-card__url'))
                wait.until(productCountPresent)
            except:
                break

        xpath2='//*[@class="product-card__url"]'
        products = driver.find_elements_by_xpath(xpath2)
        print(sub_url)
        print('Number of results', category,len(products))
        sub_urls=[]
        for p in products:
            sub_urls.append(p.get_attribute('href'))
        driver.quit()
        
        for u in sub_urls:
            try:
                url=u
                driver= webdriver.Chrome('chromedriver',options=options)
                driver.get(url)
                driver.set_window_size(1920,1080)
                name=driver.find_element_by_xpath('//*[@class="product__name"]').text
                article_nos = driver.find_element_by_name('ProductCode')
                article_no=article_nos.get_attribute("value")
                price=driver.find_element_by_xpath('//*[@class="product__price"]').text
                brand=driver.find_element_by_xpath('//*[@class="product__brand"]').text
                color=driver.find_element_by_xpath('//span[@id="colorModel-button"]').text
                img_url=driver.find_element_by_xpath('//*[@id="ProductImages"]/div/div/div[1]/div/img[1]').get_attribute('src')

    #             img_url = "https:" + img_url
                img_url=img_url.replace(' ','+')
                ac='kappah'+'/'+article_no+str('.jpg')
                print(img_url)
                opener = urllib.request.URLopener()
                opener.addheader('User-Agent', 'whatever')
                filename, headers2 = opener.retrieve(img_url, ac)


                blob_name=str('kappah')+str('/')+str(article_no)
                path_to_file='kappah/'+article_no+str('.jpg')
                blob_url=upload_to_bucket(blob_name, path_to_file)


                siz=driver.find_element_by_id('sizeModel')
                sizes=siz.find_elements_by_xpath(".//option")
                for s in sizes:
                    size=s.get_attribute('innerHTML') 
                    print(img_url,size,article_no,brand,name,color,url,price)

                    with open("kappah.csv", "a", encoding='utf-8') as f:
                        writer = csv.writer(f)
                        writer.writerow([name,brand,gender,category,size,price,color,url,img_url,article_no,blob_url])
                driver.quit()
            except:
                error.append(url)
                driver.quit()
                continue



In [5]:

scrapper(main_url='https://www.kappahl.com/sv-SE/barn/44-86/nyheter/',gender='boy')

<input type="checkbox" class="filterlabel__checkbox">
<input type="checkbox" class="filterlabel__checkbox">
https://www.kappahl.com/sv-SE/barn/44-86/shorts/
Number of results Shorts 10
https://www.kappahl.com/ProductImages/585208/585208_F.jpg?preset=ProductImageMagnified


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.kappahl.com/ProductImages/585208/585208_F.jpg?preset=ProductImageMagnified 56  585208 NEWBIE BABY Mjukisshorts Grön https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585208 99:-
https://www.kappahl.com/ProductImages/585208/585208_F.jpg?preset=ProductImageMagnified 62  585208 NEWBIE BABY Mjukisshorts Grön https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585208 99:-
https://www.kappahl.com/ProductImages/585208/585208_F.jpg?preset=ProductImageMagnified 68  585208 NEWBIE BABY Mjukisshorts Grön https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585208 99:-
https://www.kappahl.com/ProductImages/585208/585208_F.jpg?preset=ProductImageMagnified 74  585208 NEWBIE BABY Mjukisshorts Grön https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585208 99:-
https://www.kappahl.com/ProductImages/585208/585208_F.jpg?preset=ProductImageMagnified 80  585208 NEWBIE BABY Mjukisshorts Grön https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585208 99:-
https://www.kappahl.com/P

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.kappahl.com/ProductImages/583203/583203_F.jpg?preset=ProductImageMagnified 56  583203 NEWBIE BABY Hängselshorts Grå https://www.kappahl.com/sv-SE/barn/shorts/hangselshorts/583203 199:-
https://www.kappahl.com/ProductImages/583203/583203_F.jpg?preset=ProductImageMagnified 62  583203 NEWBIE BABY Hängselshorts Grå https://www.kappahl.com/sv-SE/barn/shorts/hangselshorts/583203 199:-
https://www.kappahl.com/ProductImages/583203/583203_F.jpg?preset=ProductImageMagnified 68  583203 NEWBIE BABY Hängselshorts Grå https://www.kappahl.com/sv-SE/barn/shorts/hangselshorts/583203 199:-
https://www.kappahl.com/ProductImages/583203/583203_F.jpg?preset=ProductImageMagnified 74  583203 NEWBIE BABY Hängselshorts Grå https://www.kappahl.com/sv-SE/barn/shorts/hangselshorts/583203 199:-
https://www.kappahl.com/ProductImages/583203/583203_F.jpg?preset=ProductImageMagnified 80  583203 NEWBIE BABY Hängselshorts Grå https://www.kappahl.com/sv-SE/barn/shorts/hangselshorts/583203 199:-
https://www.kap

...

https://www.kappahl.com/ProductImages/616987/616987_L.jpg?preset=ProductImageMagnified 13/14  616987 DESIGN BY KAPPAHL Sandaler Silver https://www.kappahl.com/sv-SE/barn/accessoarer/skor/sandaler/616987 74,50:- 149:- -50%
https://www.kappahl.com/ProductImages/616987/616987_L.jpg?preset=ProductImageMagnified 15/16  616987 DESIGN BY KAPPAHL Sandaler Silver https://www.kappahl.com/sv-SE/barn/accessoarer/skor/sandaler/616987 74,50:- 149:- -50%
https://www.kappahl.com/ProductImages/616987/616987_L.jpg?preset=ProductImageMagnified 17/18  616987 DESIGN BY KAPPAHL Sandaler Silver https://www.kappahl.com/sv-SE/barn/accessoarer/skor/sandaler/616987 74,50:- 149:- -50%
https://www.kappahl.com/ProductImages/616987/616987_L.jpg?preset=ProductImageMagnified 19/20  616987 DESIGN BY KAPPAHL Sandaler Silver https://www.kappahl.com/sv-SE/barn/accessoarer/skor/sandaler/616987 74,50:- 149:- -50%
https://www.kappahl.com/ProductImages/598516/598516_F.jpg?preset=ProductImageMagnified


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.kappahl.com/ProductImages/598516/598516_F.jpg?preset=ProductImageMagnified 13/14  - Ej i lager 598516 NEWBIE BABY Tossor Offwhite https://www.kappahl.com/sv-SE/barn/accessoarer/skor/tossor/598516 69,30:- 99:- -30%
https://www.kappahl.com/ProductImages/598516/598516_F.jpg?preset=ProductImageMagnified 15/16  - Ej i lager 598516 NEWBIE BABY Tossor Offwhite https://www.kappahl.com/sv-SE/barn/accessoarer/skor/tossor/598516 69,30:- 99:- -30%
https://www.kappahl.com/ProductImages/598516/598516_F.jpg?preset=ProductImageMagnified 17/18  598516 NEWBIE BABY Tossor Offwhite https://www.kappahl.com/sv-SE/barn/accessoarer/skor/tossor/598516 69,30:- 99:- -30%
https://www.kappahl.com/ProductImages/599126/599126_F.jpg?preset=ProductImageMagnified


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.kappahl.com/ProductImages/599126/599126_F.jpg?preset=ProductImageMagnified Finns i butik 599126 DESIGN BY KAPPAHL Sneakers baby Rosa https://www.kappahl.com/sv-SE/barn/accessoarer/skor/sneakers-baby/599126 49,50:- 99:- -50%
https://www.kappahl.com/ProductImages/599126/599126_F.jpg?preset=ProductImageMagnified 13/14  - Ej i lager 599126 DESIGN BY KAPPAHL Sneakers baby Rosa https://www.kappahl.com/sv-SE/barn/accessoarer/skor/sneakers-baby/599126 49,50:- 99:- -50%
https://www.kappahl.com/ProductImages/599126/599126_F.jpg?preset=ProductImageMagnified 15/16  - Ej i lager 599126 DESIGN BY KAPPAHL Sneakers baby Rosa https://www.kappahl.com/sv-SE/barn/accessoarer/skor/sneakers-baby/599126 49,50:- 99:- -50%
https://www.kappahl.com/ProductImages/599126/599126_F.jpg?preset=ProductImageMagnified 17/18  - Ej i lager 599126 DESIGN BY KAPPAHL Sneakers baby Rosa https://www.kappahl.com/sv-SE/barn/accessoarer/skor/sneakers-baby/599126 49,50:- 99:- -50%
https://www.kappahl.com/ProductImages/

In [19]:
scrapper(main_url='https://www.kappahl.com/sv-SE/barn/44-86/nyheter/',gender='girl')

<input type="checkbox" class="filterlabel__checkbox">
<input type="checkbox" class="filterlabel__checkbox">
https://www.kappahl.com/sv-SE/barn/44-86/shorts/
Number of results Shorts 8
https://www.kappahl.com/ProductImages/585166/585166_L.jpg?preset=ProductImageMagnified


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.kappahl.com/ProductImages/585166/585166_L.jpg?preset=ProductImageMagnified 56  585166 NEWBIE BABY Mjukisshorts Gul https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585166 99:-
https://www.kappahl.com/ProductImages/585166/585166_L.jpg?preset=ProductImageMagnified 62  585166 NEWBIE BABY Mjukisshorts Gul https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585166 99:-
https://www.kappahl.com/ProductImages/585166/585166_L.jpg?preset=ProductImageMagnified 68  585166 NEWBIE BABY Mjukisshorts Gul https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585166 99:-
https://www.kappahl.com/ProductImages/585166/585166_L.jpg?preset=ProductImageMagnified 74  - Ej i lager 585166 NEWBIE BABY Mjukisshorts Gul https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585166 99:-
https://www.kappahl.com/ProductImages/585166/585166_L.jpg?preset=ProductImageMagnified 80  - Ej i lager 585166 NEWBIE BABY Mjukisshorts Gul https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585166 99:-
http

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.kappahl.com/ProductImages/585174/585174_L.jpg?preset=ProductImageMagnified 56  - Ej i lager 585174 NEWBIE BABY Mjukisshorts Lila https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585174 99:-
https://www.kappahl.com/ProductImages/585174/585174_L.jpg?preset=ProductImageMagnified 62  585174 NEWBIE BABY Mjukisshorts Lila https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585174 99:-
https://www.kappahl.com/ProductImages/585174/585174_L.jpg?preset=ProductImageMagnified 68  585174 NEWBIE BABY Mjukisshorts Lila https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585174 99:-
https://www.kappahl.com/ProductImages/585174/585174_L.jpg?preset=ProductImageMagnified 74  - Ej i lager 585174 NEWBIE BABY Mjukisshorts Lila https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585174 99:-
https://www.kappahl.com/ProductImages/585174/585174_L.jpg?preset=ProductImageMagnified 80  - Ej i lager 585174 NEWBIE BABY Mjukisshorts Lila https://www.kappahl.com/sv-SE/barn/shorts/mjukisshort

...

https://www.kappahl.com/ProductImages/365684/365684_L.jpg?preset=ProductImageMagnified 00/03 M  365684 NEWBIE BABY Tossor Offwhite https://www.kappahl.com/sv-SE/barn/44-86/skor/tossor/365684 59,90:-
https://www.kappahl.com/ProductImages/365684/365684_L.jpg?preset=ProductImageMagnified 00/06 M  - Ej i lager 365684 NEWBIE BABY Tossor Offwhite https://www.kappahl.com/sv-SE/barn/44-86/skor/tossor/365684 59,90:-
https://www.kappahl.com/ProductImages/599100/599100_F.jpg?preset=ProductImageMagnified


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.kappahl.com/ProductImages/599100/599100_F.jpg?preset=ProductImageMagnified 13/14  599100 DESIGN BY KAPPAHL Sneakers baby Grön https://www.kappahl.com/sv-SE/barn/44-86/skor/sneakers-baby/599100 49,50:- 99:- -50%
https://www.kappahl.com/ProductImages/599100/599100_F.jpg?preset=ProductImageMagnified 15/16  599100 DESIGN BY KAPPAHL Sneakers baby Grön https://www.kappahl.com/sv-SE/barn/44-86/skor/sneakers-baby/599100 49,50:- 99:- -50%
https://www.kappahl.com/ProductImages/599100/599100_F.jpg?preset=ProductImageMagnified 17/18  599100 DESIGN BY KAPPAHL Sneakers baby Grön https://www.kappahl.com/sv-SE/barn/44-86/skor/sneakers-baby/599100 49,50:- 99:- -50%
https://www.kappahl.com/ProductImages/599100/599100_F.jpg?preset=ProductImageMagnified 19/20  599100 DESIGN BY KAPPAHL Sneakers baby Grön https://www.kappahl.com/sv-SE/barn/44-86/skor/sneakers-baby/599100 49,50:- 99:- -50%
https://www.kappahl.com/ProductImages/598516/598516_F.jpg?preset=ProductImageMagnified


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.kappahl.com/ProductImages/598516/598516_F.jpg?preset=ProductImageMagnified 13/14  - Ej i lager 598516 NEWBIE BABY Tossor Offwhite https://www.kappahl.com/sv-SE/barn/accessoarer/skor/tossor/598516 69,30:- 99:- -30%
https://www.kappahl.com/ProductImages/598516/598516_F.jpg?preset=ProductImageMagnified 15/16  - Ej i lager 598516 NEWBIE BABY Tossor Offwhite https://www.kappahl.com/sv-SE/barn/accessoarer/skor/tossor/598516 69,30:- 99:- -30%
https://www.kappahl.com/ProductImages/598516/598516_F.jpg?preset=ProductImageMagnified 17/18  598516 NEWBIE BABY Tossor Offwhite https://www.kappahl.com/sv-SE/barn/accessoarer/skor/tossor/598516 69,30:- 99:- -30%


In [11]:
scrapper(main_url='https://www.kappahl.com/sv-SE/barn/86-128/nyheter/',gender='boy')

<input type="checkbox" class="filterlabel__checkbox">
<input type="checkbox" class="filterlabel__checkbox">
https://www.kappahl.com/sv-SE/barn/44-86/shorts/
Number of results Shorts 10
https://www.kappahl.com/ProductImages/585208/585208_F.jpg?preset=ProductImageMagnified


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.kappahl.com/ProductImages/585208/585208_F.jpg?preset=ProductImageMagnified 56  585208 NEWBIE BABY Mjukisshorts Grön https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585208 99:-
https://www.kappahl.com/ProductImages/585208/585208_F.jpg?preset=ProductImageMagnified 62  585208 NEWBIE BABY Mjukisshorts Grön https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585208 99:-
https://www.kappahl.com/ProductImages/585208/585208_F.jpg?preset=ProductImageMagnified 68  585208 NEWBIE BABY Mjukisshorts Grön https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585208 99:-
https://www.kappahl.com/ProductImages/585208/585208_F.jpg?preset=ProductImageMagnified 74  585208 NEWBIE BABY Mjukisshorts Grön https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585208 99:-
https://www.kappahl.com/ProductImages/585208/585208_F.jpg?preset=ProductImageMagnified 80  585208 NEWBIE BABY Mjukisshorts Grön https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585208 99:-
https://www.kappahl.com/P

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.kappahl.com/ProductImages/583203/583203_F.jpg?preset=ProductImageMagnified 56  583203 NEWBIE BABY Hängselshorts Grå https://www.kappahl.com/sv-SE/barn/shorts/hangselshorts/583203 199:-
https://www.kappahl.com/ProductImages/583203/583203_F.jpg?preset=ProductImageMagnified 62  583203 NEWBIE BABY Hängselshorts Grå https://www.kappahl.com/sv-SE/barn/shorts/hangselshorts/583203 199:-
https://www.kappahl.com/ProductImages/583203/583203_F.jpg?preset=ProductImageMagnified 68  583203 NEWBIE BABY Hängselshorts Grå https://www.kappahl.com/sv-SE/barn/shorts/hangselshorts/583203 199:-
https://www.kappahl.com/ProductImages/583203/583203_F.jpg?preset=ProductImageMagnified 74  583203 NEWBIE BABY Hängselshorts Grå https://www.kappahl.com/sv-SE/barn/shorts/hangselshorts/583203 199:-
https://www.kappahl.com/ProductImages/583203/583203_F.jpg?preset=ProductImageMagnified 80  583203 NEWBIE BABY Hängselshorts Grå https://www.kappahl.com/sv-SE/barn/shorts/hangselshorts/583203 199:-
https://www.kap

...

https://www.kappahl.com/ProductImages/616987/616987_L.jpg?preset=ProductImageMagnified 13/14  616987 DESIGN BY KAPPAHL Sandaler Silver https://www.kappahl.com/sv-SE/barn/accessoarer/skor/sandaler/616987 74,50:- 149:- -50%
https://www.kappahl.com/ProductImages/616987/616987_L.jpg?preset=ProductImageMagnified 15/16  616987 DESIGN BY KAPPAHL Sandaler Silver https://www.kappahl.com/sv-SE/barn/accessoarer/skor/sandaler/616987 74,50:- 149:- -50%
https://www.kappahl.com/ProductImages/616987/616987_L.jpg?preset=ProductImageMagnified 17/18  616987 DESIGN BY KAPPAHL Sandaler Silver https://www.kappahl.com/sv-SE/barn/accessoarer/skor/sandaler/616987 74,50:- 149:- -50%
https://www.kappahl.com/ProductImages/616987/616987_L.jpg?preset=ProductImageMagnified 19/20  616987 DESIGN BY KAPPAHL Sandaler Silver https://www.kappahl.com/sv-SE/barn/accessoarer/skor/sandaler/616987 74,50:- 149:- -50%
https://www.kappahl.com/ProductImages/599126/599126_F.jpg?preset=ProductImageMagnified


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.kappahl.com/ProductImages/599126/599126_F.jpg?preset=ProductImageMagnified 13/14  - Ej i lager 599126 DESIGN BY KAPPAHL Sneakers baby Rosa https://www.kappahl.com/sv-SE/barn/accessoarer/skor/sneakers-baby/599126 49,50:- 99:- -50%
https://www.kappahl.com/ProductImages/599126/599126_F.jpg?preset=ProductImageMagnified 15/16  599126 DESIGN BY KAPPAHL Sneakers baby Rosa https://www.kappahl.com/sv-SE/barn/accessoarer/skor/sneakers-baby/599126 49,50:- 99:- -50%
https://www.kappahl.com/ProductImages/599126/599126_F.jpg?preset=ProductImageMagnified 17/18  - Ej i lager 599126 DESIGN BY KAPPAHL Sneakers baby Rosa https://www.kappahl.com/sv-SE/barn/accessoarer/skor/sneakers-baby/599126 49,50:- 99:- -50%
https://www.kappahl.com/ProductImages/599126/599126_F.jpg?preset=ProductImageMagnified 19/20  - Ej i lager 599126 DESIGN BY KAPPAHL Sneakers baby Rosa https://www.kappahl.com/sv-SE/barn/accessoarer/skor/sneakers-baby/599126 49,50:- 99:- -50%
https://www.kappahl.com/ProductImages/598516/

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.kappahl.com/ProductImages/598516/598516_F.jpg?preset=ProductImageMagnified 13/14  - Ej i lager 598516 NEWBIE BABY Tossor Offwhite https://www.kappahl.com/sv-SE/barn/accessoarer/skor/tossor/598516 69,30:- 99:- -30%
https://www.kappahl.com/ProductImages/598516/598516_F.jpg?preset=ProductImageMagnified 15/16  - Ej i lager 598516 NEWBIE BABY Tossor Offwhite https://www.kappahl.com/sv-SE/barn/accessoarer/skor/tossor/598516 69,30:- 99:- -30%
https://www.kappahl.com/ProductImages/598516/598516_F.jpg?preset=ProductImageMagnified 17/18  598516 NEWBIE BABY Tossor Offwhite https://www.kappahl.com/sv-SE/barn/accessoarer/skor/tossor/598516 69,30:- 99:- -30%


In [6]:
scrapper(main_url='https://www.kappahl.com/sv-SE/barn/86-128/nyheter/',gender='girl')

<input type="checkbox" class="filterlabel__checkbox">
<input type="checkbox" class="filterlabel__checkbox">
https://www.kappahl.com/sv-SE/barn/44-86/shorts/
Number of results Shorts 8
https://www.kappahl.com/ProductImages/585166/585166_L.jpg?preset=ProductImageMagnified


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.kappahl.com/ProductImages/585166/585166_L.jpg?preset=ProductImageMagnified 56  585166 NEWBIE BABY Mjukisshorts Gul https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585166 99:-
https://www.kappahl.com/ProductImages/585166/585166_L.jpg?preset=ProductImageMagnified 62  585166 NEWBIE BABY Mjukisshorts Gul https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585166 99:-
https://www.kappahl.com/ProductImages/585166/585166_L.jpg?preset=ProductImageMagnified 68  585166 NEWBIE BABY Mjukisshorts Gul https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585166 99:-
https://www.kappahl.com/ProductImages/585166/585166_L.jpg?preset=ProductImageMagnified 74  - Ej i lager 585166 NEWBIE BABY Mjukisshorts Gul https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585166 99:-
https://www.kappahl.com/ProductImages/585166/585166_L.jpg?preset=ProductImageMagnified 80  - Ej i lager 585166 NEWBIE BABY Mjukisshorts Gul https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585166 99:-
http

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.kappahl.com/ProductImages/585174/585174_L.jpg?preset=ProductImageMagnified 56  - Ej i lager 585174 NEWBIE BABY Mjukisshorts Lila https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585174 99:-
https://www.kappahl.com/ProductImages/585174/585174_L.jpg?preset=ProductImageMagnified 62  585174 NEWBIE BABY Mjukisshorts Lila https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585174 99:-
https://www.kappahl.com/ProductImages/585174/585174_L.jpg?preset=ProductImageMagnified 68  585174 NEWBIE BABY Mjukisshorts Lila https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585174 99:-
https://www.kappahl.com/ProductImages/585174/585174_L.jpg?preset=ProductImageMagnified 74  - Ej i lager 585174 NEWBIE BABY Mjukisshorts Lila https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585174 99:-
https://www.kappahl.com/ProductImages/585174/585174_L.jpg?preset=ProductImageMagnified 80  - Ej i lager 585174 NEWBIE BABY Mjukisshorts Lila https://www.kappahl.com/sv-SE/barn/shorts/mjukisshort

...

https://www.kappahl.com/ProductImages/365684/365684_L.jpg?preset=ProductImageMagnified 00/03 M  365684 NEWBIE BABY Tossor Offwhite https://www.kappahl.com/sv-SE/barn/44-86/skor/tossor/365684 59,90:-
https://www.kappahl.com/ProductImages/365684/365684_L.jpg?preset=ProductImageMagnified 00/06 M  - Ej i lager 365684 NEWBIE BABY Tossor Offwhite https://www.kappahl.com/sv-SE/barn/44-86/skor/tossor/365684 59,90:-
https://www.kappahl.com/ProductImages/599100/599100_F.jpg?preset=ProductImageMagnified


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.kappahl.com/ProductImages/599100/599100_F.jpg?preset=ProductImageMagnified 13/14  599100 DESIGN BY KAPPAHL Sneakers baby Grön https://www.kappahl.com/sv-SE/barn/44-86/skor/sneakers-baby/599100 49,50:- 99:- -50%
https://www.kappahl.com/ProductImages/599100/599100_F.jpg?preset=ProductImageMagnified 15/16  599100 DESIGN BY KAPPAHL Sneakers baby Grön https://www.kappahl.com/sv-SE/barn/44-86/skor/sneakers-baby/599100 49,50:- 99:- -50%
https://www.kappahl.com/ProductImages/599100/599100_F.jpg?preset=ProductImageMagnified 17/18  599100 DESIGN BY KAPPAHL Sneakers baby Grön https://www.kappahl.com/sv-SE/barn/44-86/skor/sneakers-baby/599100 49,50:- 99:- -50%
https://www.kappahl.com/ProductImages/599100/599100_F.jpg?preset=ProductImageMagnified 19/20  599100 DESIGN BY KAPPAHL Sneakers baby Grön https://www.kappahl.com/sv-SE/barn/44-86/skor/sneakers-baby/599100 49,50:- 99:- -50%
https://www.kappahl.com/ProductImages/598516/598516_F.jpg?preset=ProductImageMagnified


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.kappahl.com/ProductImages/598516/598516_F.jpg?preset=ProductImageMagnified 13/14  - Ej i lager 598516 NEWBIE BABY Tossor Offwhite https://www.kappahl.com/sv-SE/barn/accessoarer/skor/tossor/598516 69,30:- 99:- -30%
https://www.kappahl.com/ProductImages/598516/598516_F.jpg?preset=ProductImageMagnified 15/16  - Ej i lager 598516 NEWBIE BABY Tossor Offwhite https://www.kappahl.com/sv-SE/barn/accessoarer/skor/tossor/598516 69,30:- 99:- -30%
https://www.kappahl.com/ProductImages/598516/598516_F.jpg?preset=ProductImageMagnified 17/18  598516 NEWBIE BABY Tossor Offwhite https://www.kappahl.com/sv-SE/barn/accessoarer/skor/tossor/598516 69,30:- 99:- -30%


In [7]:
scrapper(main_url='https://www.kappahl.com/sv-SE/barn/134-170/nyheter/',gender='boy')

<input type="checkbox" class="filterlabel__checkbox">
<input type="checkbox" class="filterlabel__checkbox">
https://www.kappahl.com/sv-SE/barn/44-86/shorts/
Number of results Shorts 10
https://www.kappahl.com/ProductImages/585208/585208_F.jpg?preset=ProductImageMagnified


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.kappahl.com/ProductImages/585208/585208_F.jpg?preset=ProductImageMagnified 56  585208 NEWBIE BABY Mjukisshorts Grön https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585208 99:-
https://www.kappahl.com/ProductImages/585208/585208_F.jpg?preset=ProductImageMagnified 62  585208 NEWBIE BABY Mjukisshorts Grön https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585208 99:-
https://www.kappahl.com/ProductImages/585208/585208_F.jpg?preset=ProductImageMagnified 68  585208 NEWBIE BABY Mjukisshorts Grön https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585208 99:-
https://www.kappahl.com/ProductImages/585208/585208_F.jpg?preset=ProductImageMagnified 74  585208 NEWBIE BABY Mjukisshorts Grön https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585208 99:-
https://www.kappahl.com/ProductImages/585208/585208_F.jpg?preset=ProductImageMagnified 80  585208 NEWBIE BABY Mjukisshorts Grön https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585208 99:-
https://www.kappahl.com/P

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.kappahl.com/ProductImages/583203/583203_F.jpg?preset=ProductImageMagnified 56  583203 NEWBIE BABY Hängselshorts Grå https://www.kappahl.com/sv-SE/barn/shorts/hangselshorts/583203 199:-
https://www.kappahl.com/ProductImages/583203/583203_F.jpg?preset=ProductImageMagnified 62  583203 NEWBIE BABY Hängselshorts Grå https://www.kappahl.com/sv-SE/barn/shorts/hangselshorts/583203 199:-
https://www.kappahl.com/ProductImages/583203/583203_F.jpg?preset=ProductImageMagnified 68  583203 NEWBIE BABY Hängselshorts Grå https://www.kappahl.com/sv-SE/barn/shorts/hangselshorts/583203 199:-
https://www.kappahl.com/ProductImages/583203/583203_F.jpg?preset=ProductImageMagnified 74  583203 NEWBIE BABY Hängselshorts Grå https://www.kappahl.com/sv-SE/barn/shorts/hangselshorts/583203 199:-
https://www.kappahl.com/ProductImages/583203/583203_F.jpg?preset=ProductImageMagnified 80  583203 NEWBIE BABY Hängselshorts Grå https://www.kappahl.com/sv-SE/barn/shorts/hangselshorts/583203 199:-
https://www.kap

...

https://www.kappahl.com/ProductImages/616987/616987_L.jpg?preset=ProductImageMagnified 13/14  616987 DESIGN BY KAPPAHL Sandaler Silver https://www.kappahl.com/sv-SE/barn/accessoarer/skor/sandaler/616987 74,50:- 149:- -50%
https://www.kappahl.com/ProductImages/616987/616987_L.jpg?preset=ProductImageMagnified 15/16  616987 DESIGN BY KAPPAHL Sandaler Silver https://www.kappahl.com/sv-SE/barn/accessoarer/skor/sandaler/616987 74,50:- 149:- -50%
https://www.kappahl.com/ProductImages/616987/616987_L.jpg?preset=ProductImageMagnified 17/18  616987 DESIGN BY KAPPAHL Sandaler Silver https://www.kappahl.com/sv-SE/barn/accessoarer/skor/sandaler/616987 74,50:- 149:- -50%
https://www.kappahl.com/ProductImages/616987/616987_L.jpg?preset=ProductImageMagnified 19/20  616987 DESIGN BY KAPPAHL Sandaler Silver https://www.kappahl.com/sv-SE/barn/accessoarer/skor/sandaler/616987 74,50:- 149:- -50%
https://www.kappahl.com/ProductImages/598516/598516_F.jpg?preset=ProductImageMagnified


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.kappahl.com/ProductImages/598516/598516_F.jpg?preset=ProductImageMagnified 13/14  - Ej i lager 598516 NEWBIE BABY Tossor Offwhite https://www.kappahl.com/sv-SE/barn/accessoarer/skor/tossor/598516 69,30:- 99:- -30%
https://www.kappahl.com/ProductImages/598516/598516_F.jpg?preset=ProductImageMagnified 15/16  - Ej i lager 598516 NEWBIE BABY Tossor Offwhite https://www.kappahl.com/sv-SE/barn/accessoarer/skor/tossor/598516 69,30:- 99:- -30%
https://www.kappahl.com/ProductImages/598516/598516_F.jpg?preset=ProductImageMagnified 17/18  598516 NEWBIE BABY Tossor Offwhite https://www.kappahl.com/sv-SE/barn/accessoarer/skor/tossor/598516 69,30:- 99:- -30%
https://www.kappahl.com/ProductImages/599126/599126_F.jpg?preset=ProductImageMagnified


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.kappahl.com/ProductImages/599126/599126_F.jpg?preset=ProductImageMagnified Finns i butik 599126 DESIGN BY KAPPAHL Sneakers baby Rosa https://www.kappahl.com/sv-SE/barn/accessoarer/skor/sneakers-baby/599126 49,50:- 99:- -50%
https://www.kappahl.com/ProductImages/599126/599126_F.jpg?preset=ProductImageMagnified 13/14  - Ej i lager 599126 DESIGN BY KAPPAHL Sneakers baby Rosa https://www.kappahl.com/sv-SE/barn/accessoarer/skor/sneakers-baby/599126 49,50:- 99:- -50%
https://www.kappahl.com/ProductImages/599126/599126_F.jpg?preset=ProductImageMagnified 15/16  - Ej i lager 599126 DESIGN BY KAPPAHL Sneakers baby Rosa https://www.kappahl.com/sv-SE/barn/accessoarer/skor/sneakers-baby/599126 49,50:- 99:- -50%
https://www.kappahl.com/ProductImages/599126/599126_F.jpg?preset=ProductImageMagnified 17/18  - Ej i lager 599126 DESIGN BY KAPPAHL Sneakers baby Rosa https://www.kappahl.com/sv-SE/barn/accessoarer/skor/sneakers-baby/599126 49,50:- 99:- -50%
https://www.kappahl.com/ProductImages/

In [8]:
scrapper(main_url='https://www.kappahl.com/sv-SE/barn/134-170/nyheter/',gender='girl')

<input type="checkbox" class="filterlabel__checkbox">
<input type="checkbox" class="filterlabel__checkbox">
https://www.kappahl.com/sv-SE/barn/44-86/shorts/
Number of results Shorts 8
https://www.kappahl.com/ProductImages/585166/585166_L.jpg?preset=ProductImageMagnified


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.kappahl.com/ProductImages/585166/585166_L.jpg?preset=ProductImageMagnified 56  585166 NEWBIE BABY Mjukisshorts Gul https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585166 99:-
https://www.kappahl.com/ProductImages/585166/585166_L.jpg?preset=ProductImageMagnified 62  585166 NEWBIE BABY Mjukisshorts Gul https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585166 99:-
https://www.kappahl.com/ProductImages/585166/585166_L.jpg?preset=ProductImageMagnified 68  585166 NEWBIE BABY Mjukisshorts Gul https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585166 99:-
https://www.kappahl.com/ProductImages/585166/585166_L.jpg?preset=ProductImageMagnified 74  - Ej i lager 585166 NEWBIE BABY Mjukisshorts Gul https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585166 99:-
https://www.kappahl.com/ProductImages/585166/585166_L.jpg?preset=ProductImageMagnified 80  - Ej i lager 585166 NEWBIE BABY Mjukisshorts Gul https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585166 99:-
http

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.kappahl.com/ProductImages/585174/585174_L.jpg?preset=ProductImageMagnified 56  - Ej i lager 585174 NEWBIE BABY Mjukisshorts Lila https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585174 99:-
https://www.kappahl.com/ProductImages/585174/585174_L.jpg?preset=ProductImageMagnified 62  585174 NEWBIE BABY Mjukisshorts Lila https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585174 99:-
https://www.kappahl.com/ProductImages/585174/585174_L.jpg?preset=ProductImageMagnified 68  585174 NEWBIE BABY Mjukisshorts Lila https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585174 99:-
https://www.kappahl.com/ProductImages/585174/585174_L.jpg?preset=ProductImageMagnified 74  - Ej i lager 585174 NEWBIE BABY Mjukisshorts Lila https://www.kappahl.com/sv-SE/barn/shorts/mjukisshorts/585174 99:-
https://www.kappahl.com/ProductImages/585174/585174_L.jpg?preset=ProductImageMagnified 80  - Ej i lager 585174 NEWBIE BABY Mjukisshorts Lila https://www.kappahl.com/sv-SE/barn/shorts/mjukisshort

...

https://www.kappahl.com/ProductImages/365684/365684_L.jpg?preset=ProductImageMagnified 00/03 M  365684 NEWBIE BABY Tossor Offwhite https://www.kappahl.com/sv-SE/barn/44-86/skor/tossor/365684 59,90:-
https://www.kappahl.com/ProductImages/365684/365684_L.jpg?preset=ProductImageMagnified 00/06 M  - Ej i lager 365684 NEWBIE BABY Tossor Offwhite https://www.kappahl.com/sv-SE/barn/44-86/skor/tossor/365684 59,90:-
https://www.kappahl.com/ProductImages/599100/599100_F.jpg?preset=ProductImageMagnified


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.kappahl.com/ProductImages/599100/599100_F.jpg?preset=ProductImageMagnified 13/14  599100 DESIGN BY KAPPAHL Sneakers baby Grön https://www.kappahl.com/sv-SE/barn/44-86/skor/sneakers-baby/599100 49,50:- 99:- -50%
https://www.kappahl.com/ProductImages/599100/599100_F.jpg?preset=ProductImageMagnified 15/16  599100 DESIGN BY KAPPAHL Sneakers baby Grön https://www.kappahl.com/sv-SE/barn/44-86/skor/sneakers-baby/599100 49,50:- 99:- -50%
https://www.kappahl.com/ProductImages/599100/599100_F.jpg?preset=ProductImageMagnified 17/18  599100 DESIGN BY KAPPAHL Sneakers baby Grön https://www.kappahl.com/sv-SE/barn/44-86/skor/sneakers-baby/599100 49,50:- 99:- -50%
https://www.kappahl.com/ProductImages/599100/599100_F.jpg?preset=ProductImageMagnified 19/20  599100 DESIGN BY KAPPAHL Sneakers baby Grön https://www.kappahl.com/sv-SE/barn/44-86/skor/sneakers-baby/599100 49,50:- 99:- -50%
https://www.kappahl.com/ProductImages/598516/598516_F.jpg?preset=ProductImageMagnified


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://www.kappahl.com/ProductImages/598516/598516_F.jpg?preset=ProductImageMagnified 13/14  - Ej i lager 598516 NEWBIE BABY Tossor Offwhite https://www.kappahl.com/sv-SE/barn/accessoarer/skor/tossor/598516 69,30:- 99:- -30%
https://www.kappahl.com/ProductImages/598516/598516_F.jpg?preset=ProductImageMagnified 15/16  - Ej i lager 598516 NEWBIE BABY Tossor Offwhite https://www.kappahl.com/sv-SE/barn/accessoarer/skor/tossor/598516 69,30:- 99:- -30%
https://www.kappahl.com/ProductImages/598516/598516_F.jpg?preset=ProductImageMagnified 17/18  598516 NEWBIE BABY Tossor Offwhite https://www.kappahl.com/sv-SE/barn/accessoarer/skor/tossor/598516 69,30:- 99:- -30%


In [9]:
from google.cloud import storage
storage_client = storage.Client.from_service_account_json('auth.json')
bucket = storage_client.get_bucket('urbankiddo')
blob = bucket.blob('kappah.csv')
blob.upload_from_filename('kappah.csv')
blob.public_url

'https://storage.googleapis.com/urbankiddo/kappah.csv'